# Calculating Similarity

create some transformer embedded vectors, then use a cosine similarity

In [1]:
import h
import pandas as pd
pd.set_option('display.max_colwidth', None)
# use movies dataset
df = pd.read_csv('../data/imdb_top_1000.csv')#.head(10)
df[['Series_Title', 'Overview']].head(10)

,Series_Title,Overview
0,The Shawshank Redemption,"Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency."
1,The Godfather,An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.
2,The Dark Knight,"When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice."
3,The Godfather: Part II,"The early life and career of Vito Corleone in 1920s New York City is portrayed, while his son, Michael, expands and tightens his grip on the family crime syndicate."
4,12 Angry Men,A jury holdout attempts to prevent a miscarriage of justice by forcing his colleagues to reconsider the evidence.
5,The Lord of the Rings: The Return of the King,Gandalf and Aragorn lead the World of Men against Sauron's army to draw his gaze from Frodo and Sam as they approach Mount Doom with the One Ring.
6,Pulp Fiction,"The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption."
7,Schindler's List,"In German-occupied Poland during World War II, industrialist Oskar Schindler gradually becomes concerned for his Jewish workforce after witnessing their persecution by the Nazis."
8,Inception,A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O.
9,Fight Club,"An insomniac office worker and a devil-may-care soapmaker form an underground fight club that evolves into something much, much more."


In [ ]:
sentences = df['Overview']
h.compute_similarity(0, sentences)

In [ ]:
# df['similar_movies'] = pd.Series(df.index).apply(lambda x: h.compute_similarity(x, df['Overview']))
# df['similar_movies']

In [ ]:
# import pickle
# with open('similar_movies_df.p', 'wb') as fh:
#     pickle.dump(df, fh)
# # pickle.load(open('similar_movies_df.p', 'rb'))